# Tetris 

COGS 18 Final Project: Tetris

Tetris is a tile-matching puzzle game first designed by Soviet Russian game designer Alexey Leonidovich Pajitnov.



## Based on this guide created by Professor David Kosbie

https://www.cs.cmu.edu/~112/notes/notes-tetris/index.html

Author: Zijian Liu

PID: A15723355

Instructor: Professor Shannon Ellis

Spring 2019




## Project Code


In [38]:
from tkinter import *

import random

import Tetris


run(playTetris()[0]
,playTetris()[1])
# upon successful launching the game then closing it, should print "bye!"
# If it says "NameError: name 'Tk' is not defined", something went wrong
# I haven't figure out a way to fix it yet

bye!
